!pip install pymupdf gensim nltk flask


In [6]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

pdf_path = r"C:\Users\HP\OneDrive\Desktop\vlg image\vlg_image_denoising_report.pdf"
pdf_text = extract_text_from_pdf(pdf_path)


In [8]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [9]:
# Ensure you have downloaded the necessary NLTK data files
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text).lower()
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return words

preprocessed_text = preprocess_text(pdf_text)


In [11]:
len(preprocessed_text)

445

In [12]:
from gensim.models import Word2Vec

# Train Word2Vec model (or load a pre-trained model)
model = Word2Vec([preprocessed_text], vector_size=100, window=5, min_count=1, workers=4)




import torch
from transformers import BertTokenizer, BertForQuestionAnswering

def load_model():
    model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
    tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
    return model, tokenizer

In [13]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
def get_embedding(text, model):
    words = preprocess_text(text)
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if not word_vectors:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

In [15]:
def find_best_answer(question, context, model):
    question_embedding = get_embedding(question, model)
    context_sentences = context.split('.')
    best_similarity = -1
    best_sentence = None
    for sentence in context_sentences:
        sentence_embedding = get_embedding(sentence, model)
        similarity = cosine_similarity([question_embedding], [sentence_embedding])[0][0]
        if similarity > best_similarity:
            best_similarity = similarity
            best_sentence = sentence
    return best_sentence




In [16]:
question = "what is final PSNr value?"
answer = find_best_answer(question, pdf_text, model)
print("Answer:", answer)

Answer: 3698 [ loss-MSE,epoch=50]
Result & Discussions:
PSNR Score
The Final Average PSNR value obtained for the denoised images using our CNN
model is 17
